<a href="https://colab.research.google.com/github/salonisaxenaa/Hacktoberfest2022sarthak/blob/main/horse%20human%20classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2022-11-14 09:51:10--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 172.217.194.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.1’

horse-or-human.zip. 100%[===================>] 142.65M  24.0MB/s    in 7.0s    

2022-11-14 09:51:18 (20.3 MB/s) - ‘horse-or-human.zip.1’ saved [149574867/149574867]



In [26]:
import zipfile

local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [27]:
import os
image_dir=os.path.join('./horse-or-human')
trainhorse= os.path.join('./horse-or-human/horses')
trainhuman=os.path.join('./horse-or-human/humans')

In [28]:
train_horse_names = os.listdir(trainhorse)
print(train_horse_names[:10])

train_human_names = os.listdir(trainhuman)
print(train_human_names[:10])

['horse11-2.png', 'horse05-4.png', 'horse29-8.png', 'horse42-5.png', 'horse30-0.png', 'horse09-6.png', 'horse28-8.png', 'horse23-8.png', 'horse44-0.png', 'horse26-5.png']
['human01-01.png', 'human11-12.png', 'human05-05.png', 'human09-29.png', 'human16-27.png', 'human09-04.png', 'human05-18.png', 'human03-05.png', 'human09-18.png', 'human13-12.png']


In [29]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [30]:
train_data=ImageDataGenerator(rescale=1./255)
train_gen= train_data.flow_from_directory(image_dir, target_size=(300,300), batch_size=128, class_mode='binary')

Found 1027 images belonging to 2 classes.


In [31]:
validation_data=ImageDataGenerator(rescale=1./255)
validation_gen=validation_data.flow_from_directory(image_dir, target_size=(300,300), batch_size=128, class_mode='binary')

Found 1027 images belonging to 2 classes.


In [32]:
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)), tf.keras.layers.MaxPooling2D(2,2)  , tf.keras.layers.Conv2D(32, (3,3), activation='relu'), tf.keras.layers.MaxPooling2D(2,2),  tf.keras.layers.Conv2D(32, (3,3), activation='relu'), tf.keras.layers.MaxPooling2D(2,2) ,  tf.keras.layers.Flatten(), tf.keras.layers.Dense(128, activation='relu'), tf.keras.layers.Dense(1, activation='sigmoid')])

In [33]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get('accuracy')>98.5):
      print("stopp")
      self.model.stop_training=True

callbacks=myCallback()

In [35]:
model.fit(train_gen, steps_per_epoch=8, epochs=15, verbose=1)

Epoch 1/15
8/8 [==============================] - 8s 635ms/step - loss: 4.1925 - accuracy: 0.5172
Epoch 2/15
8/8 [==============================] - 7s 794ms/step - loss: 0.6010 - accuracy: 0.7775
Epoch 3/15
8/8 [==============================] - 7s 800ms/step - loss: 0.8235 - accuracy: 0.7227
Epoch 4/15
8/8 [==============================] - 6s 711ms/step - loss: 0.8315 - accuracy: 0.7697
Epoch 5/15
8/8 [==============================] - 6s 710ms/step - loss: 0.2675 - accuracy: 0.8776
Epoch 6/15
8/8 [==============================] - 6s 699ms/step - loss: 0.4985 - accuracy: 0.8543
Epoch 7/15
8/8 [==============================] - 6s 837ms/step - loss: 0.1115 - accuracy: 0.9677
Epoch 8/15
8/8 [==============================] - 6s 703ms/step - loss: 0.0576 - accuracy: 0.9833
Epoch 9/15
8/8 [==============================] - 6s 708ms/step - loss: 0.0258 - accuracy: 0.9911
Epoch 10/15
8/8 [==============================] - 7s 809ms/step - loss: 0.3095 - accuracy: 0.8887
Epoch 11/15
8/8 [==

In [37]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving download.jpg to download (1).jpg
1/1 [==============================] - 0s 16ms/step
[6.8407585e-20]
download.jpg is a horse
